Criando Mount entre o Azure Storage Account com o Databricks

In [ ]:
dbutils.fs.mount (source = "wasbs://conteiner1@storageprojeto1.blob.core.windows.net",
                 mount_point = "/mnt/blobdataset",
               extra_configs = {'fs.azure.account.key.storageprojeto1.blob.core.windows.net': dbutils.secrets.get ('mount_Scope','chaveBlob')} )

Leitura do arquivo CSV

In [ ]:
df = spark.read.csv ('/mnt/blobdataset/dataset_transicoes.csv' ,header=True)
display(df)

Script de processamento dos dados e filtragem dos clientes

In [ ]:
from pyspark.sql.functions import col, max

df = spark.read.csv ('/mnt/blobdataset/dataset_transicoes.csv' ,header=True)

#Alteração nome da coluna do dataset e convertendo a coluna "Valor" de string para numerico
df = df.withColumnRenamed("nome", "Nome_Cliente")
df = df.withColumn("Valor", df["Valor"].cast("double"))


#Filtrando os clientes que tem o maior limite e que tiveram transições acima de 4000 reais
clientes = df.filter((col("limite_credito") > 5000) & (col("valor") > 4000))
resultado = clientes.select("CPF", "Nome_Cliente", "Limite_Credito", "Valor")

#Agrupando as colunas do dataset e usando a função sum para somar os valores da coluna "Valor"
df_resultado_final = df.groupBy("CPF", "Nome_Cliente", "Limite_Credito").agg(sum("Valor").alias("Valor_Total"))

#Removendo duplicação e agrupando por Nome do cliente
df_resultado_final = df_resultado_final.distinct()
df_resultado_final = df_resultado_final.orderBy("Nome_Cliente")


display(df_resultado_final)



Conexão do Databricks com o Azure SQL Database

In [ ]:
# Conexão do Azure databricks com Azure SQL Database

server_name = "servidor-projeto.database.windows.net"
database_name = "Database_Projeto"
username = "projeto_integrador"
password = "teste@5321"

# JDBC URL para conexão com o Azure SQL Database
jdbc_url = f"jdbc:sqlserver://{server_name}:1433;database={database_name};user={username};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"


# Escrevendo o DataFrame no Azure SQL Database
df_resultado_final.write.jdbc(url=jdbc_url, table="Clientes", mode="overwrite", properties={"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"})
